In [ ]:
%pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import time
import datetime
import os

client_id=' 공백 '
client_secret=' 공백 '


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
start_time = time.time()

artist_cache = {}  

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']

total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for genre in genre_list:
    # 각 장르별로 데이터를 저장할 빈 DataFrame 생성
    temporary_df2 = pd.DataFrame()  

    for year in year_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        artist_id = []
        track_id = []  
        artist_popularity = []
        artist_followers =[]
        
        for i in range(0, 1000, 50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                
                #아티스트 정보를 요청 코드
                
                artist_info = None
                
                #print(f"탐색중인 가수 ID: {artist_id}")
                
                if t['artists'][0]['id'] in artist_cache:                    
                    artist_info = artist_cache[t['artists'][0]['id']]
                else:
                    artist_info = sp.artist(t['artists'][0]['id'])
                    artist_cache[t['artists'][0]['id']] = artist_info              
        
                artist_popularity.append(artist_info['popularity'])  
                artist_followers.append(artist_info['followers']['total'])  
                print(f"{year}_{genre}: 진행중")
                print(f"아티스트 데이터 수집 횟수 : {i+1} // 3초 대기")
                #time.sleep(3)

                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100                 
                print(f"전체 진행 상황: {progress:.2f}% ,,, {progress:.0f}개")
            
        track_df2 = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name, 'artist_popularity': artist_popularity, 'artist_followers': artist_followers,
                                  'track_id': track_id})

        temporary_df2 = pd.concat([temporary_df2, track_df2])  

    temporary_df2.drop_duplicates(subset='track_id', inplace=True)
    temporary_df2 = temporary_df2.drop(columns=['track_id'])

    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    temporary_df2.to_csv(f'/home/ubuntu/project/4.5HZ/Data/{genre}_{year}_artist_{timestamp}.csv', index=False)
    #temporary_df2.to_csv(f'./data/{genre}_total_artist_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")
